In [3]:
import requests
import constants

url = "https://api.solcast.com.au/data/forecast/radiation_and_weather?latitude=-12.456&longitude=130.845&format=json&output_parameters=air_temp,ghi,ghi10,ghi90,wind_speed_10m,wind_direction_10m&period=PT15M"

payload = {}
headers = {
  'Authorization': f'Bearer {constants.KEY_SOLCAST}'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"forecasts":[{"air_temp":31,"ghi":598,"ghi10":598,"ghi90":598,"wind_speed_10m":5.5,"wind_direction_10m":359,"period_end":"2023-10-12T06:30:00.0000000Z","period":"PT15M"},{"air_temp":31,"ghi":543,"ghi10":528,"ghi90":543,"wind_speed_10m":5.2,"wind_direction_10m":356,"period_end":"2023-10-12T06:45:00.0000000Z","period":"PT15M"},{"air_temp":31,"ghi":475,"ghi10":354,"ghi90":486,"wind_speed_10m":5,"wind_direction_10m":353,"period_end":"2023-10-12T07:00:00.0000000Z","period":"PT15M"},{"air_temp":31,"ghi":389,"ghi10":290,"ghi90":427,"wind_speed_10m":4.6,"wind_direction_10m":348,"period_end":"2023-10-12T07:15:00.0000000Z","period":"PT15M"},{"air_temp":31,"ghi":328,"ghi10":255,"ghi90":363,"wind_speed_10m":4.2,"wind_direction_10m":339,"period_end":"2023-10-12T07:30:00.0000000Z","period":"PT15M"},{"air_temp":31,"ghi":263,"ghi10":196,"ghi90":307,"wind_speed_10m":4,"wind_direction_10m":330,"period_end":"2023-10-12T07:45:00.0000000Z","period":"PT15M"},{"air_temp":31,"ghi":210,"ghi10":157,"ghi90":247

In [13]:
from solcast import forecast

res = forecast.radiation_and_weather(
    latitude=-12.45519,
    longitude=130.841366,
    output_parameters='air_temp,ghi,ghi10,ghi90,wind_speed_10m,wind_direction_10m',
    format='json',
    api_key=constants.KEY_SOLCAST
)

res.to_pandas()

,air_temp,ghi,ghi10,ghi90,wind_speed_10m,wind_direction_10m
period_end,,,,,,
2023-10-12 07:00:00+00:00,31,514,487,514,5.1,355
2023-10-12 07:30:00+00:00,31,371,301,397,4.4,344
2023-10-12 08:00:00+00:00,31,231,149,275,3.9,324
2023-10-12 08:30:00+00:00,31,128,71,156,3.4,306
2023-10-12 09:00:00+00:00,30,48,28,55,2.8,288
...,...,...,...,...,...,...
2023-10-14 05:00:00+00:00,35,877,568,904,2.8,24
2023-10-14 05:30:00+00:00,34,829,509,830,4.3,349
2023-10-14 06:00:00+00:00,33,741,445,741,5.9,325
